# Integración CoreML de RusTorch - Enlaces Python

Este notebook demuestra cómo usar la funcionalidad CoreML de RusTorch a través de enlaces Python.

## Configuración e Importaciones

In [ ]:
# Importar enlaces Python de RusTorch
try:
    import rustorch
    print(f"✅ Versión de RusTorch: {rustorch.__version__}")
    print(f"📝 Descripción: {rustorch.__description__}")
    print(f"👥 Autor: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ Error al importar RusTorch: {e}")
    print("Por favor, construya con maturin develop")
    exit(1)

import numpy as np
import platform

print(f"🖥️ Plataforma: {platform.system()} {platform.release()}")
print(f"🐍 Versión de Python: {platform.python_version()}")

## Verificar Disponibilidad de CoreML

In [ ]:
# Verificar funcionalidad CoreML
try:
    # Verificar si CoreML está disponible
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML disponible: {coreml_available}")
    
    if coreml_available:
        print("🎉 ¡CoreML está disponible!")
        
        # Obtener información del dispositivo
        device_info = rustorch.get_coreml_device_info()
        print("📱 Información del dispositivo CoreML:")
        print(device_info)
    else:
        print("⚠️ CoreML no está disponible")
        if platform.system() != "Darwin":
            print("CoreML solo está disponible en macOS")
        else:
            print("Es posible que las características de CoreML no estén habilitadas")
            
except AttributeError:
    print("❌ Funciones CoreML no encontradas")
    print("Puede que no esté construido con características CoreML")
    coreml_available = False
except Exception as e:
    print(f"❌ Error al verificar CoreML: {e}")
    coreml_available = False

## Creación de Dispositivo CoreML y Operaciones

In [ ]:
if coreml_available:
    try:
        # Crear dispositivo CoreML
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ Dispositivo CoreML creado: {device}")
        
        # Obtener información del dispositivo
        print(f"🆔 ID del dispositivo: {device.device_id()}")
        print(f"✅ Disponible: {device.is_available()}")
        print(f"💾 Límite de memoria: {device.memory_limit()} bytes")
        print(f"🧮 Límite de unidades de cómputo: {device.compute_units_limit()}")
        print(f"📚 Tamaño de caché del modelo: {device.model_cache_size()}")
        
        # Limpieza de caché
        device.cleanup_cache()
        print("🧹 Caché limpiado")
        
    except Exception as e:
        print(f"❌ Error en operación del dispositivo CoreML: {e}")
else:
    print("⚠️ Omitiendo operaciones del dispositivo ya que CoreML no está disponible")

## Configuración del Backend CoreML

In [ ]:
if coreml_available:
    try:
        # Crear configuración del backend CoreML
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ Configuración del backend: {config}")
        
        # Verificar y modificar valores de configuración
        print(f"📊 Habilitar caché: {config.enable_caching}")
        print(f"🗂️ Tamaño máximo de caché: {config.max_cache_size}")
        print(f"📈 Habilitar perfilado: {config.enable_profiling}")
        print(f"🔄 Respaldo automático: {config.auto_fallback}")
        
        # Modificar configuración
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 Configuración actualizada: {config}")
        
        # Crear backend CoreML
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 Backend CoreML: {backend}")
        print(f"✅ Backend disponible: {backend.is_available()}")
        
        # Obtener estadísticas del backend
        stats = backend.get_stats()
        print(f"📊 Estadísticas del backend: {stats}")
        print(f"   Operaciones totales: {stats.total_operations}")
        print(f"   Aciertos de caché: {stats.cache_hits}")
        print(f"   Fallos de caché: {stats.cache_misses}")
        print(f"   Operaciones de respaldo: {stats.fallback_operations}")
        print(f"   Tasa de aciertos de caché: {stats.cache_hit_rate():.2%}")
        print(f"   Tasa de respaldo: {stats.fallback_rate():.2%}")
        print(f"   Tiempo promedio de ejecución: {stats.average_execution_time_ms:.2f}ms")
        
        # Limpieza de caché
        backend.cleanup_cache()
        print("\n🧹 Caché del backend limpiado")
        
    except Exception as e:
        print(f"❌ Error en operación del backend CoreML: {e}")
else:
    print("⚠️ Omitiendo operaciones del backend ya que CoreML no está disponible")

## Operaciones Básicas de Tensores (CPU)

Para comparar con CoreML, primero realicemos operaciones básicas en CPU.

In [ ]:
try:
    # Creación y operaciones básicas de tensores
    print("🧮 Operaciones básicas de tensores (CPU)")
    
    # Crear tensores desde arrays NumPy (interfaz simplificada)
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 Forma de la Matriz A: {data_a.shape}")
    print(f"📐 Forma de la Matriz B: {data_b.shape}")
    
    # Multiplicación de matrices con NumPy (para comparación)
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ Forma del resultado matmul NumPy: {numpy_result.shape}")
    print(f"📊 Resultado (primeros elementos): {numpy_result.flatten()[:4]}")
    
    print("\n🚀 Operaciones CPU completadas")
    
except Exception as e:
    print(f"❌ Error en operación de tensor: {e}")

## Simulación de Comparación de Rendimiento

In [ ]:
import time

def benchmark_matrix_operations():
    """Comparar rendimiento con diferentes tamaños de matriz"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 Comparación de rendimiento:")
    print("Tamaño\t\tTiempo CPU (ms)\tCoreML Esperado (ms)")
    print("-" * 58)
    
    for size in sizes:
        # Medir tiempo de ejecución CPU
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # Tiempo CoreML esperado (hipotético)
        # En implementación real, usar mediciones reales del backend CoreML
        expected_coreml_time = cpu_time * 0.6  # Suposición: CoreML es 40% más rápido
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 Nota: Los tiempos de CoreML son hipotéticos. Los valores reales dependen de la implementación específica.")

## Simulación de Selección de Dispositivo

In [ ]:
def simulate_device_selection():
    """Simular selección inteligente de dispositivo"""
    
    operations = [
        ("Multiplicación de matriz pequeña", (16, 16), "CPU"),
        ("Multiplicación de matriz mediana", (128, 128), "Metal GPU"),
        ("Multiplicación de matriz grande", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("Función de activación", (32, 64, 128, 128), "Metal GPU"),
        ("Convolución pequeña", (1, 3, 32, 32), "CPU"),
        ("Convolución grande", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("Operaciones de números complejos", (128, 128), "Metal GPU"),  # CoreML no soportado
        ("Distribución estadística", (1000,), "CPU"),  # CoreML no soportado
    ]
    
    print("🎯 Simulación de selección inteligente de dispositivo:")
    print("Operación\t\t\tForma del Tensor\t\tDispositivo Seleccionado")
    print("-" * 78)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<31}\t{shape_str:<15}\t{device}")
    
    print("\n📝 Lógica de selección:")
    print("  • Operaciones pequeñas: CPU (evitar sobrecarga)")
    print("  • Operaciones medianas: Metal GPU (equilibrado)")
    print("  • Operaciones grandes: CoreML (optimizado)")
    print("  • Operaciones no soportadas: respaldo GPU/CPU")

simulate_device_selection()

## Ejemplo Práctico: Capa Simple de Red Neuronal

In [ ]:
def simulate_neural_network_layer():
    """Simular capa de red neuronal"""
    
    print("🧠 Simulación de capa de red neuronal:")
    
    # Tamaño del lote y configuración de la capa
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10 clases
    
    print(f"📊 Tamaño del lote: {batch_size}")
    print(f"🔢 Características de entrada: {input_features}")
    print(f"🧮 Características ocultas: {hidden_features}")
    print(f"🎯 Características de salida: {output_features}")
    
    # Simulación de pase hacia adelante
    steps = [
        ("Entrada → Oculta", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("Activación ReLU", f"({batch_size}, {hidden_features})", "Metal"),
        ("Oculta → Salida", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 Simulación de pase hacia adelante:")
    total_time = 0
    
    for step, shape, device in steps:
        # Tiempo de ejecución virtual (ms)
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}ms")
    
    print(f"\n⏱️ Tiempo total del pase hacia adelante: {total_time:.2f}ms")
    print(f"🚀 Rendimiento estimado: {1000/total_time:.0f} lotes/segundo")

simulate_neural_network_layer()

## Resumen y Próximos Pasos

In [ ]:
print("📋 Resumen de Integración CoreML de RusTorch:")
print()
print("✅ Elementos completados:")
print("  • Configuración del entorno Jupyter")
print("  • Creación de kernel Rust y enlaces Python")
print("  • Verificación de disponibilidad CoreML")
print("  • Gestión de dispositivos y configuración")
print("  • Estadísticas y perfilado del backend")
print("  • Selección inteligente de dispositivo")
print()
print("🚧 Desarrollo futuro:")
print("  • Implementación real de operaciones CoreML")
print("  • Evaluación comparativa de rendimiento")
print("  • Más funciones de activación y tipos de capas")
print("  • Mejoras en manejo de errores")
print("  • Optimización de memoria")
print()
print("🎯 Próximos pasos recomendados:")
print("  1. Cargar y probar modelos CoreML reales")
print("  2. Comparar rendimiento Metal y CoreML")
print("  3. Probar con flujos de trabajo de aprendizaje profundo reales")
print("  4. Evaluar en entorno de producción")

if coreml_available:
    print("\n🎉 ¡Felicidades! CoreML está disponible y todas las características pueden ser probadas.")
else:
    print("\n⚠️ CoreML no está disponible, pero las características básicas están funcionando.")
    print("   Recomendamos construir con características CoreML habilitadas en macOS.")